# Linear MDPの別のClippingについて

[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)では価値をクリップする方法で解きました．

---

**復習**

$$
\begin{aligned}
& b_h^n(s, a)=\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}\\
& \widehat{Q}_h^n(s, a)=\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\left(\theta^{\star}+\left(\widehat{\mu}_h^n\right)^{\top} \widehat{V}_{h+1}^n\right)^{\top} \phi(s, a) \\
& \widehat{V}_h^n(s)=\min \left\{\max _a \widehat{Q}_h^n(s, a), H\right\}, \quad \pi_h^n(s)=\operatorname{argmax}_a \widehat{Q}_h^n(s, a)
\end{aligned}
$$

このとき，次のOptimismが成立しています：
$$
\begin{aligned}
\hat{Q}_h^n(s, a)-Q_h^{\star}(s, a) 
\geq 
\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\phi(s, a)^{\top}\left(\widehat{\mu}_h^n-\mu_h^{\star}\right)^{\top} \widehat{V}_{h+1}^n
\geq 0
\end{aligned}
$$
が言えます．ここで最後の部分では
$$
\begin{aligned}
&\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widehat{V}_{h+1}^n\right| 
\overset{(a)}{\leq}
\sup_{V \in \mathcal{F}}\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot V \right| 
\lesssim \beta\|\phi(s, a)\|_{\left(\Lambda_h^n\right)^{-1}}\\
&\mathcal{F}=\left\{f_{w, \beta, \Lambda}:\|w\|_2 \leq L, \beta \in[0, B], \sigma_{\min }(\Lambda) \geq \lambda\right\}
\end{aligned}
$$

を使っています．(a)はcovering argumentです．

---

別のクリップだと深いことが言える気がしたので，メモしておきます．

表記：[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb)と同じ

## 別のClipping（S依存）

上のClipとは別のClippingを考えてみましょう．各ステップ$h$で，次の形式を考えます：

$$
\begin{aligned}
 \widehat{Q}_h^n(s, a)=&
r(s, a)
+ \widetilde{P}^n_h(\cdot \mid s, a) \widehat{V}_{h+1}^n
\end{aligned}
$$

ここで，

$$
\widetilde{P}^n_h(\cdot \mid s, a) = 
\argmax_{p \in \Delta(\mathcal{S})}
p^\top \widehat{V}^n_{h+1} 
\;\text{ such that }\;
\left|p^\top \widehat{V}^n_{h+1} - \left(\widehat{P}^n_h\right)^\top \widehat{V}^n_{h+1}\right| \leq b^n_h(s, a)
% - \widehat{P}^n_h(\cdot \mid s, a) \widehat{V}^n_{h+1}|\;.
$$
としました．

**よって，上のClippingをすると，$\widetilde{P}^n_h$上のモデルベースRLを解いていることになります．**
Value全体をClipしていたときと異なり，この方法はValidなMDP上での最適化問題に帰着できます．
この$\widetilde{P}_h$について，
$$
\begin{aligned}
&\left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq \left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-\widehat{P}^n_h(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| + \left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq 2\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\overset{(a)}{\leq}
2\sup_{V \in \widetilde{\mathcal{F}}}\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \underbrace{V}_{\leq (H-h)(1+\beta_{h+1})} \right| 
% \lesssim 
% \beta\|\phi(s, a)\|_{\left(\Lambda_h^n\right)^{-1}}\\
% &\widetilde{\mathcal{F}}=\left\{f_{w, \beta, \Lambda}:\|w\|_2 \leq L, \beta \in[0, B], \sigma_{\min }(\Lambda) \geq \lambda\right\}
\end{aligned}
$$
が成り立っています．
**→これダメかも！ボーナスの影響が$^H$乗されて出てきてしまう．**

$$
\begin{aligned}
\widetilde{Q}_h^n(s, a)-Q_h^{\star}(s, a) 
\geq 
\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\phi(s, a)^{\top}\left(\widehat{\mu}_h^n-\mu_h^{\star}\right)^{\top} \widehat{V}_{h+1}^n
\geq 0
\end{aligned}
$$
が言えます．ここで最後の部分では


* **注意**上の方法では$\min_{s'}\widehat{V}_{h+1}^n(s')$を計算する必要がありました．これは状態空間のサイズに依存した計算量が必要になるので，非現実的です．代わりに$0, H$みたいな感じでクリップする方法も考えましたが，これは$\widetilde{P}$上の最適方策が出てくれないので今回やりたいこととは変わってしまいます．


## 別のClipping（S依存）

上のClipとは別のClippingを考えてみましょう．各ステップ$h$で，次の形式を考えます：

$$
\begin{aligned}
v^{\min}_{h+1} =& \min_{s'}\widehat{V}_{h+1}^n(s')\; \text{ and }\; v^{\max}_{h+1} = \max_{s'}\widehat{V}_{h+1}^n(s')\\
 \widehat{Q}_h^n(s, a)=&
\beta_h \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)} + 
r(s, a)
+ \operatorname{clip}\left\{\widehat{P}^n_h(\cdot \mid s, a) \widehat{V}_{h+1}^n,\; v^{\min}_{h+1},\; v^{\max}_{h+1}\right\}
\end{aligned}
$$

これは$\widehat{P}^n_h$の変わりに，次の射影によって得られた遷移を使っているのと同じです：

$$
\widetilde{P}^n_h(\cdot \mid s, a) = 
\argmin_{p \in \Delta(\mathcal{S})}
|p^\top \widehat{V}^n_{h+1} - \widehat{P}^n_h(\cdot \mid s, a) \widehat{V}^n_{h+1}|\;.
$$
なぜなら，
$$\left\{p^\top \widehat{V}^n_{h+1}\mid p \in \Delta(\mathcal{S})\right\} = \left\{c \mid c \in [\min_{s'}\widehat{V}^n_{h+1}(s'), \max_{s'}\widehat{V}^n_{h+1}(s')]\right\}$$
と同じだからですね．

**よって，上のClippingをすると，$\widetilde{P}^n_h$上のモデルベースRLを解いていることになります．**
Value全体をClipしていたときと異なり，この方法はValidなMDP上での最適化問題に帰着できます．
この$\widetilde{P}_h$について，
$$
\begin{aligned}
&\left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq \left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-\widehat{P}^n_h(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| + \left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\leq 2\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \widetilde{V}_{h+1}^n\right| \\
&\overset{(a)}{\leq}
2\sup_{V \in \widetilde{\mathcal{F}}}\left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot \underbrace{V}_{\leq (H-h)(1+\beta_{h+1})} \right| 
% \lesssim 
% \beta\|\phi(s, a)\|_{\left(\Lambda_h^n\right)^{-1}}\\
% &\widetilde{\mathcal{F}}=\left\{f_{w, \beta, \Lambda}:\|w\|_2 \leq L, \beta \in[0, B], \sigma_{\min }(\Lambda) \geq \lambda\right\}
\end{aligned}
$$
が成り立っています．
**→これダメかも！ボーナスの影響が$^H$乗されて出てきてしまう．**

$$
\begin{aligned}
\widetilde{Q}_h^n(s, a)-Q_h^{\star}(s, a) 
\geq 
\beta \sqrt{\phi(s, a)^{\top}\left(\Lambda_h^n\right)^{-1} \phi(s, a)}+\phi(s, a)^{\top}\left(\widehat{\mu}_h^n-\mu_h^{\star}\right)^{\top} \widehat{V}_{h+1}^n
\geq 0
\end{aligned}
$$
が言えます．ここで最後の部分では


* **注意**上の方法では$\min_{s'}\widehat{V}_{h+1}^n(s')$を計算する必要がありました．これは状態空間のサイズに依存した計算量が必要になるので，非現実的です．代わりに$0, H$みたいな感じでクリップする方法も考えましたが，これは$\widetilde{P}$上の最適方策が出てくれないので今回やりたいこととは変わってしまいます．


## リグレット解析


任意の$f \in \mathbb{R}^S$について，
$$
\begin{aligned}
& \left|\left(\widetilde{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot f\right| \\
& = \left|\left(\operatorname{Proj}_{\Delta(\mathcal{S})}[\widetilde{P}_h^n(\cdot \mid s, a)]-\operatorname{Proj}_{\Delta(\mathcal{S})}[P(\cdot \mid s, a)]\right) \cdot f\right| \\
& \leq \left|\left(\widehat{P}_h^n(\cdot \mid s, a)-P(\cdot \mid s, a)\right) \cdot f\right| 
\end{aligned}
$$

を示しましょう．

**これ成立しないから駄目ですね．$f=[1, 2]$，$P=[0.5, 0.5]$, $\hat{P}=[-0.5, 1]$とすると，$\widetilde{P}=[0, 1]$であり，明らかに成立しない**